In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [205]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_main_battle_tanks_by_country')
soup = BeautifulSoup(page.content, 'lxml')

# print(soup.prettify())

In [206]:
tables=soup.findAll('table', class_='wikitable')
# tables

In [11]:
#ensure that in quantity column are ints only (without letter or symbols)
import re
pattern = re.compile('[0-9]+(,[0-9]+)?')
pattern.match('None')


In [208]:
A=[]
B=[]
C=[]
D=[]
E=[]
prev_country_name = ''

names = {
    'AMX 13': 'AMX-13',
    'Type 69/79': 'Type 69',
    'M1/M1A1/M1A2': 'M1A1',
    '\n': 'T-72M1',
    'Leopard 2A4+ and 2A4MCAN': 'Leopard 2A4',
    'Leopard 2A4/2A5': 'Leopard 2A4',
    'Leopard 2A5/A6/A6M/A7': 'Leopard 2A5',
    'Leopard 1A3/A4': 'Leopard 1A3',
    'Leopard 1A4/5': 'Leopard 1A4',
    'T72M1M': 'T-72M1'
}

countries = {
    'Congo, Democratic Republic of': 'Democratic Republic of the Congo',
    'Congo, Republic of': 'Congo',
    'Republic of China (Taiwan)': 'Taiwan',
    'Korea, North': 'North Korea',
    'Korea, South': 'South Korea',
    'Macedonia, Republic of': 'Republic of Macedonia'
    
}
#for all tables on wikipedia page
for table in tables:
    #find all tr tags to access data
    rows = table.find_all('tr')
    for row in rows:
        #find all td tags - to acess all rows of wiki tables
        cells = row.find_all('td')
        if len(cells)==5:
            #in country column find links to wikipedia country pages
            clink = (cells[3].findAll('a'))
            quantity = str(cells[2].find(text=True))
            match = pattern.match(quantity)
            
            tank_name = str(cells[1].find(text=True))
            country_name = str(cells[0].find(text=True))
            #replace tank name with equivalent provided in dict 'names' - tidying data
            if tank_name in names:
                tank_name = names[tank_name]
            #replace country name with equivalent provided in dict 'countries' - tidying data
            if country_name in countries:
                country_name = countries[country_name]
            #append lists
            if match:
                A.append(country_name)
                prev_country_name = country_name
                B.append(tank_name)
                C.append(int(''.join(match.group(0).split(','))))
                D.append(clink[0].contents[0])
                E.append(cells[4].find(text=True))
        elif len(cells)==4:
            clink = (cells[2].findAll('a'))
            quantity = str(cells[1].find(text=True))
            match = pattern.match(quantity)
            
            tank_name = str(cells[0].find(text=True))
            
            if tank_name in names:
                tank_name = names[tank_name]
                
            if match:
                A.append(prev_country_name)
                B.append(tank_name)
                C.append(int(''.join(match.group(0).split(','))))
                D.append(clink[0].contents[0])
                E.append(cells[3].find(text=True))



In [187]:
import pandas as pd
#create dataframe
df=pd.DataFrame(A,columns=['Country '])
df['Type ']=B
df['Quantity (Estimated)']=C
df['Origin ']=D
# df
print(df.to_string())

                             Country                      Type   Quantity (Estimated)         Origin 
0                         Afghanistan                      T-55                   601    Soviet Union
1                         Afghanistan                      T-62                   170    Soviet Union
2                             Algeria                      T-55                   270    Soviet Union
3                             Algeria                      T-62                   300    Soviet Union
4                             Algeria                      T-72                   325    Soviet Union
5                             Algeria                     T-90A                   300          Russia
6                              Angola                      T-55                   200    Soviet Union
7                              Angola                      T-62                    50    Soviet Union
8                              Angola                      T-72                   

In [207]:
#list of all countries available in data
countries_lst = A + D
# print(sorted(set(countries_lst)))

In [190]:
#create database
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker

from models import Alliance, Country, Tank

if __name__ == "__main__":
    engine = db.create_engine("sqlite:///main.db")

    Session = sessionmaker(bind=engine)

    session = Session()


In [191]:
#Create country table
for i in range(len(set(countries_lst))):
    country = Country(name=sorted(set(countries_lst))[i])
    session.add(country)
    session.commit()


In [209]:
#create dictionary to easily acess a country ID
dictionary = {}
for instance in session.query(Country).all():
    dictionary[instance.name] = instance.id 


In [193]:
#Create tank table
for i in range(len(A)):
    tank = Tank(name=B[i], quantity=C[i], country_id=dictionary[A[i]], origin_id=dictionary[D[i]])
    session.add(tank)
    session.commit()

In [129]:
#import alliance dataset
import os
os.getcwd()
df1 = pd.read_csv("alliance_v4.1_by_directed.csv", dtype = {"dyad_st_year": int}, usecols = ['state_name1', 'state_name2', 'dyad_st_year', 'dyad_end_year'] )
df1

,state_name1,state_name2,dyad_st_year,dyad_end_year
0,United Kingdom,Portugal,1816,NaN
1,Portugal,United Kingdom,1816,NaN
2,United Kingdom,Sweden,1816,1911.0
3,Sweden,United Kingdom,1816,1911.0
4,Hanover,Bavaria,1838,1848.0
...,...,...,...,...
6439,South Sudan,Sudan,2012,2012.0
6440,Egypt,Israel,1979,NaN
6441,Israel,Egypt,1979,NaN
6442,United States of America,Israel,1981,1991.0


In [199]:
#abandon countries which aren't in countries_lst
df_state_1 =  df1[df1['state_name1'].isin(sorted(set(countries_lst)))]
df_state_1

,state_name1,state_name2,dyad_st_year,dyad_end_year
0,United Kingdom,Portugal,1816,NaN
1,Portugal,United Kingdom,1816,NaN
2,United Kingdom,Sweden,1816,1911.0
3,Sweden,United Kingdom,1816,1911.0
40,Germany,Hanover,1838,1848.0
...,...,...,...,...
6437,Armenia,Russia,2010,NaN
6438,Sudan,South Sudan,2012,2012.0
6440,Egypt,Israel,1979,NaN
6441,Israel,Egypt,1979,NaN


In [200]:
#abandon countries which aren't in countries_lst
df_state_2 =  df_state_1[df_state_1['state_name2'].isin(sorted(set(countries_lst)))]
df_state_2

,state_name1,state_name2,dyad_st_year,dyad_end_year
0,United Kingdom,Portugal,1816,NaN
1,Portugal,United Kingdom,1816,NaN
2,United Kingdom,Sweden,1816,1911.0
3,Sweden,United Kingdom,1816,1911.0
188,United Kingdom,Germany,1816,1822.0
...,...,...,...,...
6435,Belarus,Russia,2009,NaN
6436,Russia,Armenia,2010,NaN
6437,Armenia,Russia,2010,NaN
6440,Egypt,Israel,1979,NaN


In [210]:
#leave only alliances which have no ending year - they are still valid
df_end = df_state_2[~df_state_2['dyad_end_year'].notnull()]
df_end

,state_name1,state_name2,dyad_st_year,dyad_end_year
0,United Kingdom,Portugal,1816,NaN
1,Portugal,United Kingdom,1816,NaN
1830,Mauritania,Somalia,1974,NaN
1832,Mauritania,Morocco,1973,NaN
1833,Mauritania,Algeria,1973,NaN
...,...,...,...,...
6435,Belarus,Russia,2009,NaN
6436,Russia,Armenia,2010,NaN
6437,Armenia,Russia,2010,NaN
6440,Egypt,Israel,1979,NaN


In [203]:
end_year_lst = df_end['dyad_end_year'].tolist()
country_1_lst = df_end['state_name1'].tolist()
country_2_lst = df_end['state_name2'].tolist()
start_year_lst = df_end['dyad_st_year'].tolist()

In [204]:
# Create Alliance table
for i in range(len(country_1_lst)):
    alliance = Alliance(country1_id=dictionary[country_1_lst[i]], country2_id=dictionary[country_2_lst[i]], start_year=start_year_lst[i], end_year=end_year_lst[i])
    session.add(alliance)
    session.commit()

In [211]:
session.close()